In [1]:
#POST FUNCTION TEST

import requests
import pandas as pd

#----THIS CODE IS DIRECTLY FROM https://apps.fs.usda.gov/fiadb-api DOCUMENTATION----
#This code writes the function for the API request
# this function will accept a FIADB-API fullreport URL and return dataframes for the estimates as well as subtotals, and totals where available.

def fiadb_api_POST(parameterDictionary):
    # make request
    resp = requests.post(r"https://apps.fs.usda.gov/fiadb-api/fullreport",data=parameterDictionary)
    # parse response to json
    data = resp.json()

    # create output dictionary and populate it with estimate data frames
    outDict = {}
    # append estimates
    outDict['estimates'] = pd.DataFrame(data['estimates'])

    # append subtotals and totals if present
    if 'subtotals' in data.keys():
        subT = {}
        for i in data['subtotals'].keys():
            subT[i] = pd.DataFrame(data['subtotals'][i])
        outDict['subtotals'] = subT
        outDict['totals'] = pd.DataFrame(data['totals'])

    # append metadata
    outDict['metadata'] = data['metadata']
    return outDict

In [2]:
# This code block will store variables for the API request and .csv save formatting

#Colorado - 2019
statename = 'Colorado'
statecode = '082019'

# statename = 'Alabama'
# statecode = '12022'

In [3]:
# The following code blocks will make the API requests for each estimate type

# 79_SAMPLEDLANDWATER_ACRES
slw_parameters = {'snum':'79', 'wc':statecode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
sampledlandwater_acres_GET = fiadb_api_POST(slw_parameters)

In [4]:
# 2_FORESTLAND_ACRES
fl_parameters = {'snum':'2', 'wc':statecode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
forestland_acres_GET = fiadb_api_POST(fl_parameters)

In [5]:
# 20_NETSAWLOGVOLSAWTMBR_BRDFT
nslv_parameters = {'snum':'20', 'wc':statecode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
nslv_brdft_GET = fiadb_api_POST(nslv_parameters)

In [6]:
# 14_NETMERCHBOLELIVETREES_CBFT
nmblt_parameters = {'snum':'14', 'wc':statecode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
nmblt_cbft_GET = fiadb_api_POST(nmblt_parameters)

In [7]:
# 203_AVGNETGROWTHSAWTMBR_BRDFT
angst_parameters = {'snum':'203', 'wc':statecode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
angst_brdft_GET = fiadb_api_POST(angst_parameters)

In [8]:
# Average annual gross growth of sawlog volume of sawtimber trees, in board feet
# 1203_AVGANNGROSSGROWTHSAWTMBR_BRDFT
aaggst_parameters = {'snum':'1203', 'wc':statecode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
aaggst_brdft_GET = fiadb_api_POST(aaggst_parameters)

In [9]:
# 227_AVGREMOVALSAWTMBR_BRDFT
arst_parameters = {'snum':'227', 'wc':statecode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
arst_brdft_GET = fiadb_api_POST(arst_parameters)

In [10]:
# 98_CARBONLIVEABOVEGRND_MT
clab_parameters = {'snum':'98', 'wc':statecode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
clab_mt_GET = fiadb_api_POST(clab_parameters)

In [11]:
# This code block will format the returned data for each estimate and join them BY COUNTY, and export the final .csv
# 79_SAMPLEDLANDWATER_ACRES
sampledlandwater_acres = sampledlandwater_acres_GET['estimates']
slw_acres = sampledlandwater_acres[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
slw_79_acres = slw_acres.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '79_SAMPLEDLANDWATER_ACRES'})

# 2_FORESTLAND_ACRES
forestland_acres = forestland_acres_GET['estimates']
fl_acres = forestland_acres[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
fl_2_acres = fl_acres.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '2_FORESTLAND_ACRES'})

# 20_NETSAWLOGVOLSAWTMBR_BRDFT
nslv_brdft = nslv_brdft_GET['estimates']
nslv_brdft_temp = nslv_brdft[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
nslv_20_brdft = nslv_brdft_temp.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '20_NETSAWLOGVOLSAWTMBR_BRDFT'})

# 14_NETMERCHBOLELIVETREES_CBFT
nmblt_cbft = nmblt_cbft_GET['estimates']
nmblt_cbft_temp = nmblt_cbft[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
nmblt_14_cbft = nmblt_cbft_temp.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '14_NETMERCHBOLELIVETREES_CBFT'})

# 203_AVGNETGROWTHSAWTMBR_BRDFT
angst_brdft = angst_brdft_GET['estimates']
angst_brdft_temp = angst_brdft[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
angst_203_brdft = angst_brdft_temp.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '203_AVGNETGROWTHSAWTMBR_BRDFT'})

# 1203_AVGANNGROSSGROWTHSAWTMBR_BRDFT
aaggst_brdft = aaggst_brdft_GET['estimates']
aaggst_brdft_temp = aaggst_brdft[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
aaggst_1203_brdft = aaggst_brdft_temp.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '1203_AVGANNGROSSGROWTHSAWTMBR_BRDFT'})

# 227_AVGREMOVALSAWTMBR_BRDFT
arst_brdft = arst_brdft_GET['estimates']
arst_brdft_temp = arst_brdft[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
arst_227_brdft = arst_brdft_temp.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '227_AVGREMOVALSAWTMBR_BRDFT'})

# 98_CARBONLIVEABOVEGRND_MT
clab_mt = clab_mt_GET['estimates']
clab_mt_temp = clab_mt[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
clab_98_mt = clab_mt_temp.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '98_CARBONLIVEABOVEGRND_MT'})

In [ ]:
# Inner Join on dataframes
state_data = slw_79_acres.merge(
    fl_2_acres, how='left', on='STATE_COUNTY_CODE').merge(
    nslv_20_brdft, how='left', on='STATE_COUNTY_CODE').merge(
    nmblt_14_cbft, how='left', on='STATE_COUNTY_CODE').merge(
    angst_203_brdft, how='left', on='STATE_COUNTY_CODE').merge(
    aaggst_1203_brdft, how='left', on='STATE_COUNTY_CODE').merge(
    arst_227_brdft, how='left', on='STATE_COUNTY_CODE').merge(
    clab_98_mt, how='left', on='STATE_COUNTY_CODE')

state_data_temp = state_data.rename(columns={'SE_x': 'SE', 'SE_PERCENT_x': 'SE_PERCENT', 'PLOT_COUNT_x': 'PLOT_COUNT', 'SE_y': 'SE', 'SE_PERCENT_y': 'SE_PERCENT', 'PLOT_COUNT_y': 'PLOT_COUNT'})

state_data_final = state_data_temp.fillna(0)

In [13]:
state_data_final.to_csv('./state_data_csv_files/' + statename + '_' + statecode + '.csv', index=False)